# Initial Exploration of Mimic III data
Two goals in this notebook:
  * identify which set of patients I want to include in my study 
  * identify the set of variables that I want to include

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
from google.oauth2 import service_account

In [3]:
!dir ..

 Volume in drive C is OS
 Volume Serial Number is 4A63-CF1F

 Directory of C:\Users\Austin Bell\Documents\Columbia\Advanced Big Data\Project

09/02/2020  16:47    <DIR>          .
09/02/2020  16:47    <DIR>          ..
09/02/2020  16:46    <DIR>          explore
09/02/2020  16:47             2,327 Patient-Similarity-credentials.json
               1 File(s)          2,327 bytes
               3 Dir(s)  69,246,603,264 bytes free


In [14]:
# apply credentials
credentials = service_account.Credentials.from_service_account_file('../Patient-Similarity-credentials.json')
pandas_gbq.context.credentials = credentials
pandas_gbq.context.project = "patient-similarity"

In [18]:
test_query = "select * from  `patient-similarity.mimic.admissions` limit 15"
pandas_gbq.read_gbq(test_query)

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,8902,7275,136809,2139-08-28 12:35:00,2139-09-01 16:32:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,EPISCOPALIAN,DIVORCED,WHITE,2139-08-28 10:38:00,2139-08-28 14:30:00,ALTERED MENTAL STATUS; DYSPNEA,0,1
1,21467,17575,187131,2124-10-21 08:32:00,2124-11-01 11:14:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Government,None,OTHER,SINGLE,BLACK/AFRICAN AMERICAN,2124-10-21 05:04:00,2124-10-21 09:52:00,MVA;TRAUMA,0,1
2,30863,25251,142346,2167-01-23 16:23:00,2167-02-06 13:53:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,None,EPISCOPALIAN,DIVORCED,WHITE,2167-01-23 12:54:00,2167-01-23 18:00:00,INFECTED RIGHT FOOT,0,1
3,39212,32339,181355,2110-07-29 23:56:00,2110-08-04 15:10:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,JEWISH,MARRIED,WHITE,2110-07-29 20:17:00,2110-07-30 01:24:00,GASTROINTESTINAL BLEED;CHEST PAIN;TELEMETRY,0,1
4,37441,30746,155603,2131-08-22 23:07:00,2131-08-28 13:10:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,JEWISH,DIVORCED,WHITE,2131-08-22 14:15:00,2131-08-23 01:20:00,ALTERED MENTAL STATUS,0,1
5,40230,41420,143206,2147-01-06 16:57:00,2147-01-08 18:30:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,JEWISH,WIDOWED,WHITE,2147-01-06 13:30:00,2147-01-06 17:36:00,ACUTE SUBDURAL HEMATOMA,0,1
6,43547,51820,148131,2184-02-06 17:37:00,2184-02-08 15:23:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,JEWISH,SINGLE,WHITE,2184-02-06 16:18:00,2184-02-06 19:57:00,UPPER GI BLEED,0,1
7,39656,32702,190719,2156-02-19 23:51:00,2156-02-23 15:40:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,JEWISH,MARRIED,WHITE,2156-02-19 21:19:00,2156-02-20 00:45:00,S/P FALL,0,1
8,45890,58899,177890,2102-04-04 08:12:00,2102-04-10 16:00:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,NOT SPECIFIED,SINGLE,WHITE,2102-04-04 06:46:00,2102-04-04 08:33:00,THREATENED LEFT LOWER EXTREMITY,0,1
9,44975,56165,135230,2179-07-03 16:34:00,2179-07-09 12:45:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,ICF,Medicare,ENGL,NOT SPECIFIED,SEPARATED,WHITE,2179-07-03 11:30:00,2179-07-03 18:51:00,? TB,0,1


## Identify subset of patients to work with
I think I am going to focus on patients who received at least one lymphoma diagnoses

In [32]:
# identify lymphoma patients
q1 = """SELECT distinct subject_id FROM `patient-similarity.mimic.diagnoses_icd` 
where  icd9_code in ('07022', '07032', '07033', '07044','07054') 
or ICD9_CODE in ('4560', '4561', '45620', '45621')
or ICD9_CODE in ('5710', '5712', '5713','5715', '5716', '5718', '5719', '5723', '5728', '5735', 'V427')  
or icd9_code between '57140' and '57149' """
liver_pts = pandas_gbq.read_gbq(q1).subject_id.tolist()
liver_pts[:5], len(liver_pts)

# 2884 patients - this is a good number I think

([1024, 2304, 6912, 76032, 11008], 2884)

In [33]:
# is there a high mortality rate? important if I want to conduct survival analyses
q2 = """SELECT * FROM `patient-similarity.mimic.patients` 
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
and dod is not null"""
pandas_gbq.read_gbq(q2)
# 1,505 patients with a recorded date of death

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,14477,15309,F,2096-08-13,2176-01-24,NaT,2176-01-24,1
1,30059,32012,F,2058-04-26,2144-02-08,2144-02-08,2144-02-08,1
2,7627,8060,F,2111-11-26,2164-02-08,2164-02-08,2164-02-08,1
3,33557,50556,F,2110-03-06,2164-02-16,2164-02-16,2164-02-16,1
4,7860,8309,F,2060-02-13,2116-02-24,2116-02-24,2116-02-24,1
...,...,...,...,...,...,...,...,...
1500,22194,23508,M,2090-01-27,2171-09-23,NaT,2171-09-23,1
1501,11181,11819,M,2059-10-18,2111-10-23,2111-10-23,2111-10-23,1
1502,29217,31086,M,2083-01-14,2139-10-23,NaT,2139-10-23,1
1503,32080,44996,M,2061-12-27,2131-12-15,NaT,2131-12-15,1


In [34]:
# what is the average number of hospital visits for lymph patients
q3 = """SELECT avg(num_visits) as avg_visits 
from (select subject_id, count(HADM_ID) as num_visits FROM `patient-similarity.mimic.admissions` 
where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`)
group by SUBJECT_ID )
"""
pandas_gbq.read_gbq(q3)
# time series over visits might not be the most useful
# would only be able to do it over a small group of patients

,avg_visits
0,1.688974


## What are the Clinically Relevant Variables that I want to include?
Exploration into what type of data we have

In [35]:
# What kind of chart events do we have?
q4 = """SELECT ITEMID, LABEL , LINKSTO   FROM `patient-similarity.mimic.d_items`
where ITEMID in (select itemid from `patient-similarity.mimic.chartevents`
  where SUBJECT_ID in (select subject_id from `patient-similarity.mimic.liver_pts`))"""
pandas_gbq.read_gbq(q4)
# okay we have about 2772 potential chart events to include

,ITEMID,LABEL,LINKSTO
0,503,PCV Set Insp. Press,chartevents
1,25,AV Interval,chartevents
2,111,CT #4 [Fluctuate],chartevents
3,403,Inc #4 [Dressing],chartevents
4,1898,pantoprazole,chartevents
...,...,...,...
2767,224917,Impaired Skin Length #3,chartevents
2768,227753,Dialysis Catheter Placement Confirmed by X-ray,chartevents
2769,227952,Position Change,chartevents
2770,224753,Temporary Atrial Sens Setting mV,chartevents


In [37]:
# lets see how often these all occur or see if there is a way to potentially combine some of them?
q5 = """SELECT A.ITEMID, A.LABEL , B.num_occurences  FROM `patient-similarity.mimic.d_items` as A
inner join (select itemid, count(subject_id) as num_occurences from `patient-similarity.mimic.chartevents`
            where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`) 
            group by itemid) as B
on a.itemid=b.itemid
order by num_occurences desc """
pandas_gbq.read_gbq(q5)
# There is some clear overlap between the chart events - I will likely need to combine chart events
# maybe I can find some available cleaning code for this, otherwise I would have to do it myself
# additionally, I need to find a study that identifies important lymphoma chart events 

,ITEMID,LABEL,num_occurences
0,220045,Heart Rate,345818
1,220210,Respiratory Rate,343019
2,220277,O2 saturation pulseoxymetry,335922
3,211,Heart Rate,324282
4,742,calprevflg,322119
...,...,...,...
2767,227038,PHPaCO2Score_ApacheIV,1
2768,227519,Urine output_ApacheIV,1
2769,226750,Chronic Dilaysis,1
2770,225736,Triple Introducer Line Tip Cultured,1


In [38]:
# Lab events
q6 = """SELECT A.ITEMID, A.LABEL , B.num_occurences  FROM `patient-similarity.mimic.d_labitems` as A
inner join (select itemid, count(subject_id) as num_occurences from `patient-similarity.mimic.labevents`
            where subject_id in (select subject_id from `patient-similarity.mimic.liver_pts`) 
            group by itemid) as B
on a.itemid=b.itemid
order by num_occurences desc"""
pandas_gbq.read_gbq(q6)

,ITEMID,LABEL,num_occurences
0,51221,Hematocrit,123780
1,50971,Potassium,115298
2,50983,Sodium,111790
3,51265,Platelet Count,111382
4,50912,Creatinine,110742
...,...,...,...
671,51311,CD16,1
672,51316,CD22,1
673,51322,CD38,1
674,51329,CD57,1


In [40]:
# Microbiology events - not quite sure why I have to do this double group by
q7 = """SELECT SPEC_TYPE_DESC, count(SPEC_TYPE_DESC ) as num_items  FROM `patient-similarity.mimic.microbiologyevents` 
where SUBJECT_ID in (select subject_id from `patient-similarity.mimic.liver_pts` )
group by SPEC_TYPE_DESC 
order by num_items desc"""
pandas_gbq.read_gbq(q7)

,SPEC_TYPE_DESC,num_items
0,BLOOD CULTURE,27246
1,URINE,15777
2,SPUTUM,10041
3,SWAB,4449
4,STOOL,4268
...,...,...
62,Blood (Malaria),2
63,Touch Prep/Sections,1
64,BLOOD,1
65,WORM,1


In [41]:
# procedure events
# d_procedures contains the descriptions
# procedures_icd identifies which patient had which procedure and the order of the procedures
# procedureevents identifies the results of those procedures
q8 = """SELECT A.ICD9_CODE, A.short_title, B.num_procedures 
FROM `patient-similarity.mimic.d_icd_procedures`  as A
inner join (select ICD9_CODE, count(distinct SUBJECT_ID  ) as num_procedures from `patient-similarity.mimic.procedures_icd`  
  where SUBJECT_ID in (select SUBJECT_ID from `patient-similarity.mimic.liver_pts`)
  group by ICD9_CODE) as B
on A.ICD9_CODE = B.ICD9_CODE 
order by num_procedures desc"""
pandas_gbq.read_gbq(q8)

,ICD9_CODE,short_title,num_procedures
0,3893,Venous cath NEC,1266
1,9604,Insert endotracheal tube,910
2,5491,Percu abdominal drainage,748
3,966,Entral infus nutrit sub,741
4,9671,Cont inv mec ven <96 hrs,722
...,...,...,...
938,2811,Tonsil&adenoid biopsy,1
939,9979,Other therapeu apheresis,1
940,5372,Opn abd diaphrm hern NEC,1
941,9981,Hypothermia,1


There are way too many events for me to even have an idea of where to start.  Therefore, I need to read some research to find the key events for lymphoma (key lab events, chart events, procedures, and microbiology events)